In [1]:
%load_ext autoreload
%autoreload 2

import numpy as np
import math
import matplotlib.pyplot as plt
import scanpy as sc
import anndata as ad
import pandas as pd
import anndata

In [2]:
path = './data/hippo'

# Data preprocessing

In [3]:
data_slice1= sc.read_visium(f'{path}/hippo-visium')
crd = data_slice1.obsm["spatial"]

x = 4510
y = 3830
r = 2100

keep = []
for ii in range(len(crd)):
    r2 = (crd[ii, 0] - x) ** 2 + (crd[ii, 1] - y) ** 2
    if r2 <= r**2:
        keep.append(ii)
keep = np.array(keep)
data_slice1 = data_slice1[keep, :]
df1_location = data_slice1.obsm['spatial']
obs1=data_slice1.obs_names
var1=data_slice1.var_names
data_slice1 = anndata.AnnData(X=np.array(data_slice1.X.todense()))
data_slice1.obs_names=obs1
data_slice1.var_names=var1
data_slice1.obsm["spatial"] = df1_location

E:\ProgramFiles\anaconda3\envs\py39\lib\site-packages\anndata\_core\anndata.py:1908: UserWarning: Variable names are not unique. To make them unique, call `.var_names_make_unique`.
  utils.warn_names_duplicates("var")
E:\ProgramFiles\anaconda3\envs\py39\lib\site-packages\anndata\_core\anndata.py:1908: UserWarning: Variable names are not unique. To make them unique, call `.var_names_make_unique`.
  utils.warn_names_duplicates("var")


In [4]:
d2 = pd.read_csv(f'{path}/Puck_191204_01_bead_locations.csv')
df2_location=d2[["xcoord","ycoord"]]
df2_location=np.array(df2_location)
df2_barcode_names=pd.DataFrame(index=d2["barcodes"])

data=open(f'{path}/Puck_191204_01.digital_expression.txt')
df2_barcode=data.readline()
df2_barcode=data.readline()
df2_barcode=data.readline()
df2_barcode=data.readline()
df2=data.readlines()
data.close()
print(1)

df2_gene_names=[]
for i in range(df2.__len__()):
    df2[i]=df2[i].split("\t")
    df2_gene_names.append(df2[i][0])
    del(df2[i][0])
    df2[i]=[int(x) for x in df2[i]]
print(1)

df2=np.matrix(df2)
df2=df2.T
df2=np.array(df2)
df2_gene_names=pd.DataFrame(index=df2_gene_names)

data_slice2 = anndata.AnnData(X=df2, obs=df2_barcode_names,var=df2_gene_names)
data_slice2.obsm["spatial"] = df2_location

1
1


In [5]:
d3 = pd.read_csv(f'{path}/Puck_200115_08_bead_locations.csv')
df3_location=d3[["xcoord","ycoord"]]
df3_location=np.array(df3_location)
df3_barcode_names=pd.DataFrame(index=d3["barcodes"])

data=open(f'{path}/Puck_200115_08.digital_expression.txt')
df3_barcode=data.readline()
df3=data.readlines()
data.close()
print(1)

df3_gene_names=[]
for i in range(df3.__len__()):
    df3[i]=df3[i].split("\t")
    df3_gene_names.append(df3[i][0])
    del(df3[i][0])
    df3[i]=[int(x) for x in df3[i]]
print(1)

df3=np.matrix(df3)
df3=df3.T
df3=np.array(df3)
df3_gene_names=pd.DataFrame(index=df3_gene_names)

data_slice3 = anndata.AnnData(X=df3, obs=df3_barcode_names,var=df3_gene_names)
data_slice3.obsm["spatial"] = df3_location

1
1


In [6]:
data_slice3=data_slice3[data_slice3.obsm['spatial'][:,0]>800]
data_slice3=data_slice3[data_slice3.obsm['spatial'][:,1]>700]
data_slice3=data_slice3[data_slice3.obsm['spatial'][:,1]<1.5*data_slice3.obsm['spatial'][:,0]+3000]
data_slice3=data_slice3[data_slice3.obsm['spatial'][:,1]>data_slice3.obsm['spatial'][:,0]-4000]
data_slice3=data_slice3[data_slice3.obsm['spatial'][:,1]<0.5*data_slice3.obsm['spatial'][:,0] + 4250]

data_slice2=data_slice2[data_slice2.obsm['spatial'][:,1]>3000-data_slice2.obsm['spatial'][:,0]]
data_slice2=data_slice2[data_slice2.obsm['spatial'][:,1]<4000+data_slice2.obsm['spatial'][:,0]]
data_slice2=data_slice2[data_slice2.obsm['spatial'][:,1]>700]

In [7]:
def process_data(adata, n_top_genes=2000):
    adata.var_names_make_unique()
    adata.var["mt"] = adata.var_names.str.match("^MT|^mt|^Mt")
    sc.pp.calculate_qc_metrics(adata, qc_vars=["mt"], inplace=True)
    sc.pp.filter_cells(adata, min_counts=500) 
    sc.pp.filter_genes(adata, min_cells=10)
    adata.layers["counts"] = adata.X.copy()
    sc.pp.normalize_total(adata, inplace=True)
    sc.pp.log1p(adata)
    sc.pp.highly_variable_genes(
        adata, flavor="seurat", n_top_genes=n_top_genes, subset=True
    )
    return adata

data_slice1 = process_data(data_slice1, n_top_genes=3000)
data_slice2 = process_data(data_slice2, n_top_genes=3000)
data_slice3 = process_data(data_slice3, n_top_genes=3000)

In [8]:
data_slice1_index=[]
data_slice2_index=[]
data_slice3_index=[]
for i in range(data_slice1.shape[1]):
    if(data_slice1.var_names[i] in data_slice2.var_names):
        if(data_slice1.var_names[i] in data_slice3.var_names):
            data_slice1_index.append(i)
            data_slice2_index.append(np.where(data_slice2.var_names==data_slice1.var_names[i])[0][0])
            data_slice3_index.append(np.where(data_slice3.var_names==data_slice1.var_names[i])[0][0])

data_slice1=data_slice1[:,data_slice1_index]
data_slice2=data_slice2[:,data_slice2_index]
data_slice3=data_slice3[:,data_slice3_index]

In [9]:
sc.tl.pca(data_slice1, svd_solver='arpack')
sc.tl.pca(data_slice2, svd_solver='arpack')
sc.tl.pca(data_slice3, svd_solver='arpack')

E:\ProgramFiles\anaconda3\envs\py39\lib\site-packages\scanpy\preprocessing\_pca.py:229: ImplicitModificationWarning: Setting element `.obsm['X_pca']` of view, initializing view as actual.
  adata.obsm['X_pca'] = X_pca
E:\ProgramFiles\anaconda3\envs\py39\lib\site-packages\scanpy\preprocessing\_pca.py:229: ImplicitModificationWarning: Setting element `.obsm['X_pca']` of view, initializing view as actual.
  adata.obsm['X_pca'] = X_pca
E:\ProgramFiles\anaconda3\envs\py39\lib\site-packages\scanpy\preprocessing\_pca.py:229: ImplicitModificationWarning: Setting element `.obsm['X_pca']` of view, initializing view as actual.
  adata.obsm['X_pca'] = X_pca


## Using Prefast to get the embedding

In [10]:
data_slice1_X=pd.DataFrame(data_slice1.layers["counts"])
data_slice2_X=pd.DataFrame(data_slice2.layers["counts"])
data_slice3_X=pd.DataFrame(data_slice3.layers["counts"])
data_slice1_X.index=data_slice1.obs_names
data_slice2_X.index=data_slice2.obs_names
data_slice3_X.index=data_slice3.obs_names
data_slice1_X.columns=data_slice1.var_names
data_slice2_X.columns=data_slice2.var_names
data_slice3_X.columns=data_slice3.var_names

pd.DataFrame(data_slice1_X).to_csv(f'{path}/data_slice1_X.csv',header=True, index=True)
pd.DataFrame(data_slice2_X).to_csv(f'{path}/data_slice2_X.csv',header=True, index=True)
pd.DataFrame(data_slice3_X).to_csv(f'{path}/data_slice3_X.csv',header=True, index=True)
pd.DataFrame(data_slice1.obsm['spatial']).to_csv(f'{path}/data_slice1_spatial.csv',header=False, index=False)
pd.DataFrame(data_slice2.obsm['spatial']).to_csv(f'{path}/data_slice2_spatial.csv',header=False, index=False)
pd.DataFrame(data_slice3.obsm['spatial']).to_csv(f'{path}/data_slice3_spatial.csv',header=False, index=False)

In [11]:
data_slice1.write_h5ad(f'{path}/data_slice1.h5ad')
data_slice2.write_h5ad(f'{path}/data_slice2.h5ad')
data_slice3.write_h5ad(f'{path}/data_slice3.h5ad')